In [1]:
import numpy as np
import pandas as pd

In [9]:
donations = pd.read_csv('https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/donations.csv', parse_dates=['timestamp'])
donations.head(3)

,amount,timestamp,user
0,25.0,2017-11-12 11:13:44,0.0
1,50.0,2015-08-25 19:01:45,0.0
2,25.0,2015-03-26 12:03:47,0.0


In [10]:
emails = pd.read_csv('https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/emails.csv', parse_dates=['week'])
emails.head(3)

,emailsOpened,user,week
0,3.0,1.0,2015-06-29
1,2.0,1.0,2015-07-13
2,2.0,1.0,2015-07-20


In [11]:
year_joined = pd.read_csv('https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/year_joined.csv', parse_dates=['yearJoined'])
year_joined.head(3)

,user,userStats,yearJoined
0,0,silver,2014-01-01
1,1,silver,2015-01-01
2,2,silver,2016-01-01


In [6]:
year_joined.groupby('user').count().groupby('userStats').count()

,yearJoined
userStats,
1,1000


### Verify & clean emailsOpened column

In [7]:
emails[emails.emailsOpened < 1]

,emailsOpened,user,week


There are two possibilities: either nulls are not reported or members always have at least one email event. Anyone who has worked with email data knows that it’s difficult to get people to open emails, so the hypothesis that members always open at least one email per week is quite unlikely. In this case, we can resolve this by looking at the history of just one user:

In [8]:
emails[emails.user == 998]

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04 00:00:00
25465,3.0,998.0,2017-12-11 00:00:00
25466,3.0,998.0,2017-12-18 00:00:00
25467,3.0,998.0,2018-01-01 00:00:00
25468,3.0,998.0,2018-01-08 00:00:00
25469,2.0,998.0,2018-01-15 00:00:00
25470,3.0,998.0,2018-01-22 00:00:00
25471,2.0,998.0,2018-01-29 00:00:00
25472,3.0,998.0,2018-02-05 00:00:00
25473,3.0,998.0,2018-02-12 00:00:00


We can see that some weeks are missing. There aren’t any December 2017 email events after December 18, 2017.

We can check this more mathematically by calculating how many weekly observations we should have between the first and last event for that member. First we calculate the length of the member’s tenure, in weeks

In [13]:
# get the number of weeks the user opened the emails + 1
(max(emails[emails.user == 998].week) -
 min(emails[emails.user == 998].week)).days / 7

25.0

In [15]:
emails[emails.user == 998].shape

(24, 3)

Here we see 24 columns out of 26. There are non-recorded missing values